In [45]:
import pylab
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [46]:
flow = pd.read_csv("flow.tsv", na_values=['-'], delimiter="\t", error_bad_lines=False)
occupancy = pd.read_csv("occupancy.tsv", na_values=['-'], delimiter="\t", error_bad_lines=False)
speed = pd.read_csv("speed.tsv", na_values=['-'], delimiter="\t", error_bad_lines=False)
timestamp = pd.read_csv("timestamp.tsv", na_values=['-'], delimiter="\t", error_bad_lines=False)

In [ ]:
flow.head()

In [47]:
df = pd.concat([flow,occupancy,speed,timestamp], axis=1)

In [49]:
# len(df)

3073384

In [48]:
df.columns = ['flow1','flow2','flow3','flow4','occupancy1','occupancy2', 'occupancy3','occupancy4','speed1','speed2', 'speed3','speed4','timestamp']
df.head()

flow1  flow2  flow3  flow4  occupancy1  occupancy2  occupancy3  occupancy4  \
0   35.0   15.0   29.0   35.0        23.0         5.0        20.0        24.0   
1   29.0   23.0   28.0   43.0        24.0         8.0        24.0        27.0   
2   25.0    5.0   24.0   26.0        19.0         1.0        19.0        15.0   
3   31.0   15.0   25.0   37.0        26.0         5.0        18.0        27.0   
4   31.0   16.0   27.0   40.0        22.0         5.0        26.0        24.0   

   speed1  speed2  speed3  speed4            timestamp  
0    52.2    50.6    49.8    48.0  2009-08-19T15:06:15  
1    61.2    55.2    48.0    54.0  2009-08-19T15:07:14  
2    57.6    55.2    54.6    51.0  2009-08-19T15:08:14  
3    49.2    60.0    54.0    49.2  2009-08-19T15:09:14  
4    46.2    64.2    42.6    48.0  2009-08-19T15:10:15

In [50]:
df1 =  pd.DataFrame({"flow" : df.flow1, "occupancy" : df.occupancy1,"speed" : df.speed1,"timestamp" : df.timestamp,"detector" : "x"})
df2 = pd.DataFrame({"flow" : df.flow2, "occupancy" : df.occupancy2,"speed" : df.speed2,"timestamp" : df.timestamp,"detector" : "y"})
df3 = pd.DataFrame({"flow" : df.flow3, "occupancy" : df.occupancy3,"speed" : df.speed3,"timestamp" : df.timestamp,"detector" : "z"})
df4 = pd.DataFrame({"flow" : df.flow4, "occupancy" : df.occupancy4,"speed" : df.speed4,"timestamp" : df.timestamp,"detector" : "x1"})

In [51]:
frames = [df1, df2, df3, df4]
result = pd.concat(frames)

In [52]:
result = result.sort_values("timestamp")

In [53]:
len(result)

12293536

In [ ]:
# Finding outlier values for Occoupancy
val = np.sort(result["occupancy"].unique())
plt.plot(val)

# so keeping occupancy threshold as 120 since all values above them show a steep change indicating possible outliers.
# jam_occupancy_threshold = 120

In [ ]:
print np.sort(result["occupancy"].unique())

In [ ]:
# Outlier speed visualization
plt.figure(figsize=(10,10))
# plt.xlim(550, 650)
plt.plot(np.sort(result["occupancy"].unique()))

In [ ]:
# result[(pd.isnull(result['speed'])) | (pd.isnull(result['occupancy'])) | (pd.isnull(result['flow']))]

In [54]:
removeResult1 = result[(result['speed'] < 0) | (result['flow'] < 0)]
removeResult2 = result[(result['flow'] == 0) & (result['speed'] > 0)]
jam_occupancy_threshold = 100
removeResult3 = result[((result['speed'] == 0) & 
                            (result['flow'] == 0) & 
                            (result['occupancy'] != 0) & 
                            (result['occupancy'] < jam_occupancy_threshold))]
# skewed data
removeResult4 = result[(result['speed'] == 0) & (result['flow'] == 0) & (result["occupancy"] == 0)]
removeResult5 = result[(result['speed'] == 0) & (result['flow'] != 0) & (result["occupancy"] == 0)]
removeResult6 = result[(pd.isnull(result['speed'])) | (pd.isnull(result['occupancy'])) | (pd.isnull(result['flow']))]
# removeResult7 = result[result['occupancy'] > jam_occupancy_threshold] # occupancy increases rapidly after 80, mostly outliers

In [56]:
len(result)

11929404

In [55]:
result = pd.concat([result,removeResult1,removeResult1]).drop_duplicates(keep=False)
result = pd.concat([result,removeResult2,removeResult2]).drop_duplicates(keep=False)
result = pd.concat([result,removeResult3,removeResult3]).drop_duplicates(keep=False)
result = pd.concat([result,removeResult4,removeResult4]).drop_duplicates(keep=False)
result = pd.concat([result,removeResult5,removeResult5]).drop_duplicates(keep=False)
result = pd.concat([result,removeResult6,removeResult6]).drop_duplicates(keep=False)
# result = pd.concat([result,removeResult7,removeResult7]).drop_duplicates(keep=False)

In [57]:
result = result.sort_values(["flow"])

In [58]:
def sampleData(data, threshold, columnName):
    flowBins = np.zeros(shape=(20,), dtype = object)
    data = data.sort_values(columnName)
    difference = threshold
    high  = difference
    low = 0;
    for i in range(0, 20, 1):
        b = data[data[columnName] < high]
        b = b[b[columnName] >= low]
        flowBins[i] = b
        low = high
        high = high + difference
    return flowBins

In [59]:
print len(result)

11929404


In [60]:
flowBins = np.zeros(shape=(20,), dtype = object)
flowBins = sampleData(result, 13 ,"flow")    

In [61]:

flow_occupancy_bin = np.ndarray(shape=(20,20), dtype = object)
for i in range(0,20,1):
    data = pd.DataFrame(flowBins[i])
    flow_occupancy_bin[i] = sampleData(data, 6 ,"occupancy") 

In [62]:
flow_occupancy_speed_bin = np.ndarray(shape=(20,20,20), dtype = object)
for i in range(0,20,1):
    for j in range(0,20,1):
        data = flow_occupancy_bin[i][j];
        flow_occupancy_speed_bin[i][j] = sampleData(data, 11 ,"speed") 

In [63]:
!mkdir output

In [64]:
final_result = pd.DataFrame(columns = ["detector","flow","occupancy","speed","timestamp", "probability"])
total_points = len(result)
#final_result = ["0" , "0", "0", "0"]
for i in range(0,20,1):
    for j in range(0,20,1):
        for k in range(0,20,1):
            prob_data = flow_occupancy_speed_bin[i][j][k]
            points_inside_box = prob_data.shape[0]
            f = prob_data.flow.max() - prob_data.flow.min()
            o = prob_data.occupancy.max() - prob_data.occupancy.min()
            s = prob_data.speed.max() - prob_data.speed.min()
            #volume = f*o*s
            volume = 858
            if(volume == 0): volume =1
            probability = points_inside_box/float(total_points * volume)
#             print(prob_data)
            prob_data["probability"] = probability
            prob_data.to_csv("output/out" + str(i) + '-' + str(j) + '-' + str(k) + ".csv",sep="\t")
#             print(prob_data.head(1))
#           final_result.add([prob_data['flow'], prob_data['speed'], prob_data['occupancy'], prob_data['probability']])
            #final_result_align, prob_data_align = final_result.align(prob_data,axis=1)
            #final_result = pd.concat([final_result_align, prob_data_align])
            #final_result = np.append(final_result, [prob_data['flow'], prob_data['speed'], prob_data['occupancy'], prob_data['probability']])

In [65]:
! cat output/*.csv > merged.csv

In [66]:
!rm -rf output/

In [67]:
merged = pd.read_csv("merged.csv", na_values=['-'], delimiter="\t", error_bad_lines=False)

/Users/abhisek/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (2,3,4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [68]:
removeHeader = merged[merged["probability"] == "probability"]

In [69]:
merged = pd.concat([merged,removeHeader,removeHeader]).drop_duplicates(keep=False)

In [70]:
merged

Unnamed: 0 detector   flow occupancy         speed  \
0          2857611.0       x1    6.0       1.0           0.0   
1          2714264.0        z    7.0       5.0           0.0   
2          2744528.0        z    4.0       2.0           0.0   
3          2995739.0       x1    4.0       2.0           0.0   
4           294627.0        y    8.0       3.0           0.2   
5           414833.0        z    3.0       2.0           0.4   
6           847407.0        y   10.0       4.0           0.4   
7          1568976.0        y    3.0       1.0           0.5   
8           414858.0       x1    3.0       2.0           0.7   
9            47222.0       x1    6.0       3.0           1.0   
10          241428.0        y    2.0       4.0           1.1   
11         2024720.0        y   10.0       4.0           1.4   
12          504366.0        y    5.0       4.0           1.4   
13         1230755.0        y    1.0       1.0           1.4   
14         1783647.0        x    2.0       1.0           1.5   
15         2348456.0        y    1.0       1.0           1.5   
16         2245040.0        z    3.0       2.0           1.6   
17         2682855.0        y    3.0       2.0   1.767756288   
18         2467987.0        y    7.0       3.0           1.8   
19           53015.0        y    1.0       1.0           1.8   
20         1130683.0        y   12.0       4.0           2.1   
21          241385.0       x1    3.0       3.0           2.2   
22         1852267.0        y    9.0       3.0           2.3   
23          515504.0       x1    5.0       3.0           2.3   
24         1112352.0        y    2.0       1.0           2.4   
25         2450990.0       x1    4.0       2.0           2.4   
26          755106.0        z    9.0       5.0           2.5   
27         2472324.0        z    6.0       4.0           2.6   
28          795014.0        y    1.0       1.0           2.8   
29          171121.0        z    3.0       4.0           2.8   
...              ...      ...    ...       ...           ...   
11935783   2816361.0        y   80.0       7.0  149.12908608   
11936033    717640.0       x1   79.0      20.0          51.6   
11936035   2056541.0       x1   85.0      23.0          56.4   
11936036    597502.0       x1   79.0      21.0          57.0   
11936037   1639045.0       x1   80.0      19.0          57.6   
11936038   2754513.0       x1   79.0      23.0  59.651634432   
11936058   1775406.0        z   80.0      26.0          45.0   
11936059    304462.0       x1   79.0      27.0          51.0   
11936060   1770047.0       x1   78.0      27.0          52.8   
11936061    297232.0       x1   86.0      28.0          54.0   
11936063    239518.0       x1   78.0      27.0          55.2   
11936064    324649.0        z   81.0      27.0          55.8   
11936082   1511204.0       x1   80.0      33.0          28.8   
11936084    736564.0        x   78.0      30.0          35.4   
11936086   2527377.0        x   79.0      33.0          47.4   
11936087   3060484.0        x   79.0      30.0  50.952437744   
11936088     49443.0        z   79.0      32.0          51.0   
11936089   2096992.0       x1   80.0      33.0          51.0   
11936090   1264357.0       x1   80.0      32.0          52.8   
11936177   2712625.0       x1  103.0       4.0           0.0   
11936198   2719316.0       x1   91.0       9.0           0.0   
11936200   2758222.0        y   97.0       7.0  16.155650992   
11936453   1033725.0       x1   94.0      22.0          43.8   
11936461   2725036.0       x1   96.0      24.0           0.0   
11936496    187743.0       x1   93.0      35.0          52.2   
11936583   2669794.0       x1  116.0       3.0           0.0   
11936989   2988118.0        x  123.0       1.0  149.12908608   
11937006   2761107.0        y  125.0       7.0  16.155650992   
11937011   2767306.0        y  118.0       7.0  149.12908608   
11937012   2967064.0        y  123.0       7.0  149.12908608   

                    timestamp        prob

In [71]:
# merged = merged.reset_index(drop=True)
merged["probability"] = merged["probability"].apply(pd.to_numeric)
merged = merged.sort_values(["probability"])
final_columns = ['flow', 'speed', 'occupancy', 'probability']
merged = merged[final_columns]

In [ ]:
merged

flow          speed occupancy   probability
11599367   44.0          108.3      11.0  9.769987e-11
11934719   65.0           66.0       5.0  9.769987e-11
7400889    18.0  191.964169192      27.0  9.769987e-11
7461050    25.0   149.12908608      32.0  9.769987e-11
5607779    24.0    69.42056788      65.0  9.769987e-11
5607777    25.0           60.8      61.0  9.769987e-11
5607545    21.0   149.12908608      63.0  9.769987e-11
3769197    24.0   158.44965396      10.0  9.769987e-11
11922102   50.0   91.254044368      38.0  9.769987e-11
7485313    24.0           94.4      32.0  9.769987e-11
7504519    24.0   149.12908608      39.0  9.769987e-11
7764530    34.0           22.0      76.0  9.769987e-11
3525649    13.0          180.5       0.0  9.769987e-11
7651557    32.0    19.33026324       4.0  9.769987e-11
11935283   65.0   27.340332448      27.0  9.769987e-11
11935271   77.0   21.588324168      27.0  9.769987e-11
11935266   65.0   78.292770192      20.0  9.769987e-11
7764597    29.0           16.3      90.0  9.769987e-11
7764622    26.0          144.0     100.0  9.769987e-11
7521569    14.0   85.467756288      36.0  9.769987e-11
7553330    24.0   149.12908608      44.0  9.769987e-11
3522807     8.0   56.340332448      48.0  9.769987e-11
7648355   166.0   29.825817216       7.0  9.769987e-11
11596172   39.0           92.6       5.0  9.769987e-11
11925753   52.0           83.3       5.0  9.769987e-11
11925762   58.0   149.12908608       7.0  9.769987e-11
11934569   52.0    63.26170364      44.0  9.769987e-11
11614365   39.0   110.39196688      21.0  9.769987e-11
11926268   55.0          115.5      14.0  9.769987e-11
11492495   28.0   157.32908608      43.0  9.769987e-11
...         ...            ...       ...           ...
1184157       6             57         3  1.537039e-04
1184158       8             57         4  1.537039e-04
1184148       6             57         3  1.537039e-04
1184109       8             57         4  1.537039e-04
1184135       9             57         3  1.537039e-04
1184133       7             57         5  1.537039e-04
1184111       7             57         3  1.537039e-04
1184112       8             57         4  1.537039e-04
1184113      10             57         4  1.537039e-04
1184114       8             57         3  1.537039e-04
1184115       8             57         4  1.537039e-04
1184116       6             57         5  1.537039e-04
1184117       7             57         3  1.537039e-04
1184118      10             57         4  1.537039e-04
1184119      10             57         4  1.537039e-04
1184120      12             57         5  1.537039e-04
1184134       8             57         3  1.537039e-04
1184121       6             57         3  1.537039e-04
1184123       8             57         3  1.537039e-04
1184124       8             57         3  1.537039e-04
1184125       8             57         5  1.537039e-04
1184126       8             57         4  1.537039e-04
1184127       8             57         4  1.537039e-04
1184128       8             57         3  1.537039e-04
1184129       8             57         4  1.537039e-04
1184130       8             57         3  1.537039e-04
1184131      12             57         5  1.537039e-04
1184132       8             57         3  1.537039e-04
1184122       8             57         4  1.537039e-04
1997056       8           61.8         5  1.537039e-04

[11929388 rows x 4 columns]

In [ ]:
# merged
final_result = pd.DataFrame(columns = ["flow","speed","occupancy", "probability"])
for i in range(0,len(merged),100):
    final_result = final_result.append(merged.iloc[[i]])

In [ ]:
final_result.to_csv("3445.txt",sep="\t", index=False)

In [ ]:
final_result.ta()

In [ ]:
# len(merged)